# How Bitcoin's price action affects Altcoin markets

## Holy grail of crypto trading

Trading cryptocurrencies most profitably means switching effectively between holding altcoins, Bitcoin and FIAT(regular currency). This analysis aims to better understand altcoins behaviour based on Bitcoin's price action, since the correlation with Bitcoin is most relevant fact to lean on.

Historically, altcoin markets tend to rise together. In crypto community, such period is often called "altseason". During that time, most of the altcoins experience parabolic growth followed by equally rapid decline. After altseason prices eventually tend to visit previous bottoms again, accumulation zone. 

Analysis in steps:
- observe time periods based on market cycles, 
- plot Bitcoin daily chart with historical events,
- mark days when altcoin markets experienced highest volatility. 

As the indicator for altcoins volatility we:
- calculate daily returns (percent price change compared to previous day's close) calculated for selected coins,
- identify median value, rather than mean value. This approach will exclude price action of outlier coins with abnormal volatility.

Altcoins tend to pump during whole altseason unexpectedly. So the days when whole markets dumps due to correlation with Bitcoin, should be more frequent.

### Data preparation

Bitcoin's data is loaded from Bitstamp exchange on daily timeframe. Altcoins data is loaded from Bittrex exchange due to availabity of historical data. We load 25+ coins equally spread by size of current market cap with minimum 2 years history, resample data to 1D timeframe and calculate daily returns.

Analysis will cover interval from March 2017 to February 2019.

Our altcoins sample group: ARDR, DASH, DGB, DOGE, ETC, ETH, FCT, GAME, GRS, KMD, LSK, LTC, MONA, NAV, PIVX, STEEM, STRAT, UBQ, VIA, WAVES, XCP, XEM, XLM, XMR, XRP, XVG, ZCL, ZEC

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import requests
import bs4
import re
import talib

pd.set_option("display.precision", 8)
pd.set_option("display.max_rows", 999)

# Set date range for whole analysis
start_hist = '2017-3-1'
end_hist = '2019-4-18'

# Load Bitcoin data
btc_ohlcv_init = pd.read_csv('csv/BTCUSD_BITS.csv', index_col=['timeStart'],
                        parse_dates=True, usecols=['timeStart', 'open', 'high', 'low', 'close', 'volume'])
btc_ohlcv = btc_ohlcv_init.resample('D').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'})
btc_ohlcv['volume'] = btc_ohlcv_init['volume'].resample('D').sum()

# Calculate indicators
btc_ohlcv['sma50'] = talib.SMA(btc_ohlcv['close'], timeperiod=50)
btc_ohlcv['sma200'] = talib.SMA(btc_ohlcv['close'], timeperiod=200)

# Altcoins sample group
altcoins = ['ARDR', 'DASH', 'DGB', 'DOGE', 'ETC', 'ETH', 'FCT', 'GAME', 'GRS', 'KMD',
            'LSK', 'LTC', 'MONA', 'NAV', 'PIVX', 'STEEM', 'STRAT', 'UBQ', 'VIA', 'WAVES',
            'XCP','XEM', 'XLM', 'XMR', 'XRP', 'XVG', 'ZCL', 'ZEC']

df_dict = {}
for item in altcoins:
    item += 'BTC'
    df = pd.read_csv('csv/' + item + '.csv', index_col=['timeStart'],
                     parse_dates=True, usecols=['timeStart', 'close'])
    df = df.resample('D').last()
    df['pct_change'] = df.pct_change()*100
    df = df.loc[start_hist:end_hist]
    df_dict[item] = df  

#### Get current marketcaps of selected altcoins

 - **high-caps**, more than 1 billion
 - **mid-caps**, more than 50 million and less than 1 billion
 - **low-caps**, less than 50 million

In [2]:
mcaps = {}
for i in range(1,10):
    url = 'https://coinmarketcap.com/' + str(i)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    for row in soup.findAll('table')[0].tbody.findAll('tr'):
        symbol = row.select('td')[5].getText().strip()
        symbol = re.sub("[^A-Z]", "", symbol)
        if symbol in altcoins:
            mcap = row.select('td')[2].getText().strip()
            mcap = float(re.sub("[^0-9]", "", mcap))/1000000
            mcaps[symbol] = mcap

high_caps = dict((k, v) for k, v in mcaps.items() if v >= 1000)
mid_caps = dict((k, v) for k, v in mcaps.items() if v < 1000 and v >= 50)
low_caps = dict((k, v) for k, v in mcaps.items() if v < 50)

print('Altcoins sample group sorted by marketcap in millions:\n')
for k, v  in mcaps.items():
    print(k + ':',round(v, 2))

print('\nHigh caps:', str(len(high_caps)),'\nMid caps:',str(len(mid_caps)),'\nLow caps:',str(len(low_caps)),'\n')

Altcoins sample group sorted by marketcap in millions:

ETH: 16449.2
XRP: 12353.82
LTC: 4463.21
XLM: 1903.04
XMR: 1069.99
DASH: 972.12
ETC: 591.0
XEM: 523.46
ZEC: 382.53
DOGE: 294.46
WAVES: 220.08
LSK: 204.03
DGB: 128.37
XVG: 122.13
KMD: 108.88
STEEM: 106.76
STRAT: 83.4
FCT: 79.94
ARDR: 66.26
MONA: 62.49
PIVX: 36.2
GRS: 25.97
NAV: 11.94
VIA: 10.3
ZCL: 9.79
UBQ: 9.43
GAME: 7.34
XCP: 4.9

High caps: 5 
Mid caps: 15 
Low caps: 8 



#### Get Bitcoin's historical events

We scraped events from __[here](https://99bitcoins.com/price-chart-history/)__.

In [3]:
df_events = pd.read_csv('btc_events_b.csv', index_col=['date'], parse_dates=True)
df_events.sort_index(inplace=True)
df_events

,events
date,
2016-07-09,Second Halving
2016-08-02,Bitfinex Hacked
2016-11-09,"Donald Trump Elected as President, Market Plummet"
2017-03-10,SEC denies Winkelvos Bitcoin ETF
2017-03-28,SEC denies second Bitcoin ETF
2017-08-01,Bitcoin Cash hard fork
2017-09-03,China bans companies from raising money throug...
2017-09-15,China is shutting down all Bitcoin and Crypto ...
2017-10-31,CME announces to launch Bitcoin futures


Prepare dataframe with all altcoin's daily percent change and calculate median values.

In [4]:
df_returns = pd.DataFrame()
for k, v in df_dict.items():
    df_returns[k[:-3]] = v['pct_change']
df_returns['median'] = df_returns.median(axis=1)

In [5]:
def high_vol_days(max, min):
    df_down = df_returns[df_returns['median'] < min]['median']
    df_down = pd.merge(df_down.to_frame(), btc_ohlcv['low'].to_frame(), left_index=True, right_index=True)
    df_down['median_str'] = df_down['median'].apply(lambda x: str(round(x)) + '%')
    df_up = df_returns[df_returns['median'] > max]['median']
    df_up = pd.merge(df_up.to_frame(), btc_ohlcv['low'].to_frame(), left_index=True, right_index=True)
    df_up['median_str'] = df_up['median'].apply(lambda x: '+' + str(round(x)) + '%')
    
    return df_down, df_up

In [6]:
# df_down = df_returns[df_returns['median'] < -6]['median']
# df_down = pd.merge(df_down.to_frame(), btc_ohlcv['low'].to_frame(), left_index=True, right_index=True)
# df_down['median_str'] = df_down['median'].apply(lambda x: str(round(x)) + '%')
# df_up = df_returns[df_returns['median'] > 6]['median']
# df_up = pd.merge(df_up.to_frame(), btc_ohlcv['low'].to_frame(), left_index=True, right_index=True)
# df_up['median_str'] = df_up['median'].apply(lambda x: '+' + str(round(x)) + '%')

# print('Total days in observed period (2017-3-1 - 2019-2-28) with median daily returns:\n\nHigher than +8%:'
#       ,len(df_up),'days\n')

# print(df_up['median_str'])
      
# print('\nLower than -8%:', len(df_down),'days\n')

# print(df_down['median_str'])

In [7]:
from bokeh.models import Range1d
from bokeh.plotting import figure, show,  output_notebook, gridplot
output_notebook()
from dateutil import parser
import datetime
import talib

Loading BokehJS ...

#### Plot historical median daily returns of altcoin markets

In [8]:
start_hist_dt  = parser.parse(start_hist) 
end_hist_dt = parser.parse(end_hist) 
p_daily_returns =  figure(tools="save", x_axis_type="datetime", plot_width=800, plot_height=300,
               title=None, x_range=Range1d(start=start_hist_dt, end=end_hist_dt))
p_daily_returns.line(df_returns.index, df_returns['median'], line_width=1)
p_daily_returns.yaxis.axis_label = 'Daily returns in %'
show(p_daily_returns)

In [9]:
df_returns['median_smoothed'] = df_returns['median'].rolling(window=21, center=True).mean()
p_daily_returns_smoothed =  figure(tools="save", x_axis_type="datetime", plot_width=800, plot_height=300,
               title=None, x_range=Range1d(start=start_hist_dt, end=end_hist_dt))
p_daily_returns_smoothed.line(df_returns.index, df_returns['median_smoothed'], line_width=1)
p_daily_returns_smoothed.yaxis.axis_label = 'Daily returns in %'
show(p_daily_returns_smoothed)

#### Define fuctions for plotting

Since we need only 4 plots, making separate function for each will be fastest way.

In [10]:
def plot_btc_1(start, end):
    
    plot_width = 1200
    
    plot_height=500
    vol_height=100
    rsi_height=150
    
    df = btc_ohlcv.loc[start:end]
    start_dt  = parser.parse(start) 
    end_dt = parser.parse(end) 
    
    inc = df.close > df.open
    dec = df.open > df.close

    TOOLS = "save"  
    
    title = "Bitcoin 1D, from " + start + " to " + end 
    p = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height,
               title=title, x_range=Range1d(start=start_dt, end=end_dt), y_range=(0.7*df['low'].min(), 1.02*df['high'].max()))
    p.xaxis.visible = False
    p.min_border_top = 40
    
    # Add indicators
    p.line(df.index, df['sma50'], line_width=1, color='#66c2ff')
    p.line(df.index, df['sma200'], line_width=1, color='#ff6666')

    # Plot - Price chart
    p.toolbar_location = None
    p.segment(df.index, df.high, df.index, df.low, color="#808080")
    p.vbar(df.index[inc], 24 * 60 * 60 * 600, df.open[inc], df.close[inc], fill_color="#35a746",
           line_color="#35a746")
    p.vbar(df.index[dec], 24 * 60 * 60 * 600, df.open[dec], df.close[dec], fill_color="#d91813",
           line_color="#d91813")

    # Add marks for extreme volatile days  
    df_down, df_up = high_vol_days(6, -6)
    
    day_range = (end_dt - start_dt).days
    font_size = str(round(1600/day_range)) + 'pt'
    for index, row in df_down.iterrows():
        p.triangle(x=index, y=0.92*row['low'], size=1000/day_range,
                   line_color="#ff0000", line_width=1, fill_color='#ff0000')
        p.text(x=index, y=0.92*row['low'] - 200, text=[row['median_str']], x_offset=1200/day_range,
               text_font_size=font_size, angle=1.57, text_color="#ff0000")
    for index, row in df_up.iterrows():
        p.triangle(x=index, y=0.92*row['low'], size=1000/day_range,
                   line_color="#00e600", line_width=1, fill_color='#00cc00')
        p.text(x=index, y=0.92*row['low'] - 200, text=[row['median_str']], x_offset=1200/day_range,
               text_font_size=font_size, angle=1.57, text_color="#00cc00") 
        
    # Add marks for historical events
    for index, row in df_events.iterrows():
        p.line(x=[index,index], y=[0.5*df['low'].min(), 2*df['high'].max()], color='#808080')
        offset = 1400
        if 'Cash' in row['events']:
            offset = 2200
        p.text(x=index, y=df['high'].max() - offset, text=[row['events']], x_offset=3200/day_range,
           text_font_size='10pt', angle=1.57, text_color="#808080")
        #if index = ''

    # Plot - Volume chart
    p_vol = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=vol_height,
               title=None, x_range=Range1d(start=start_dt, end=end_dt))
    p_vol.yaxis.visible = False
    p_vol.quad(bottom=0, top=df.volume, left=df.index, right=df.index)
    p_vol.ygrid.grid_line_color = None
    p_vol.toolbar_location = None
    
    # Plot - Altcoins daily returns smoothed
    df_returns_range = df_returns.loc[start:end]
    p_alts_smoothed = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns SMOOTHED", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(-7,5))
#                     y_range=(df_returns['median'].rolling(window=5, center=True).mean().min(),
#                              df_returns['median'].rolling(window=5, center=True).mean().max()))
    p_alts_smoothed.line(df_returns.index, df_returns['median'].rolling(window=5, center=True).mean(), line_width=1)
    p_alts_smoothed.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts_smoothed.toolbar_location = None
    
    # Plot - Altcoins daily returns
    df_returns_range = df_returns.loc[start:end]
    p_alts = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(df_returns_range['median'].min(), df_returns_range['median'].max()))
    p_alts.line(df_returns.index, df_returns['median'], line_width=1)
    #p_alts.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts.xaxis.visible = False
    
    p_alts.yaxis.axis_label = 'Daily returns in %'
    p_alts_smoothed.yaxis.axis_label = 'Daily returns in %'
    
    grid = gridplot([[p_alts], [p_alts_smoothed], [p], [p_vol]])
    
    return grid

In [11]:
start = '2017-3-1'
end = '2017-8-4'
grid = plot_btc_1(start, end)
show(grid)

In [12]:
def plot_btc_2(start, end):
    
    plot_width = 1200
    
    plot_height=700
    vol_height=100
    rsi_height=150
    
    df = btc_ohlcv.loc[start:end]
    start_dt  = parser.parse(start) 
    end_dt = parser.parse(end) 
    
    inc = df.close > df.open
    dec = df.open > df.close

    TOOLS = "save"  
    
    title = "Bitcoin 1D, from " + start + " to " + end 
    p = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height,
               title=title, x_range=Range1d(start=start_dt, end=end_dt), y_range=(0.4*df['low'].min(), 1.02*df['high'].max()))
    p.xaxis.visible = False
    p.min_border_top = 40
    
    # Add indicators
    p.line(df.index, df['sma50'], line_width=1, color='#66c2ff')
    p.line(df.index, df['sma200'], line_width=1, color='#ff6666')

    # Plot - Price chart
    p.toolbar_location = None
    p.segment(df.index, df.high, df.index, df.low, color="#808080")
    p.vbar(df.index[inc], 24 * 60 * 60 * 600, df.open[inc], df.close[inc], fill_color="#35a746",
           line_color="#35a746")
    p.vbar(df.index[dec], 24 * 60 * 60 * 600, df.open[dec], df.close[dec], fill_color="#d91813",
           line_color="#d91813")

    # Add marks for extreme volatile days
    df_down, df_up = high_vol_days(8, -8)
    
    day_range = (end_dt - start_dt).days
    font_size = str(round(1600/day_range)) + 'pt'
    for index, row in df_down.iterrows():
        p.triangle(x=index, y=0.92*row['low'], size=1000/day_range,
                   line_color="#ff0000", line_width=1, fill_color='#ff0000')
        p.text(x=index, y=0.92*row['low'] - 1000, text=[row['median_str']], x_offset=1600/day_range,
               text_font_size=font_size, angle=1.57, text_color="#ff0000")
    for index, row in df_up.iterrows():
        p.triangle(x=index, y=0.92*row['low'], size=1000/day_range,
                   line_color="#00e600", line_width=1, fill_color='#00cc00')
        p.text(x=index, y=0.92*row['low'] - 1000, text=[row['median_str']], x_offset=1600/day_range,
               text_font_size=font_size, angle=1.57, text_color="#00cc00") 
        
    # Add marks for historical events
    for index, row in df_events.iterrows():
        p.line(x=[index,index], y=[0.2*df['low'].min(), 2*df['high'].max()], color='#808080')
        offset = 12000
        if 'CME' in row['events']:
            offset = 9000
        if 'SegWit' in row['events']:
            offset = 9000
        if 'CBOE' in row['events']:
            offset = 18000
        if 'Korea' in row['events']:
            offset = 18000
        p.text(x=index, y=df['high'].max() - offset, text=[row['events']], x_offset=3200/day_range,
           text_font_size='10pt', angle=1.57, text_color="#808080")
        #if index = ''

    # Plot - Volume chart
    p_vol = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=vol_height,
               title=None, x_range=Range1d(start=start_dt, end=end_dt))
    p_vol.yaxis.visible = False
    p_vol.quad(bottom=0, top=df.volume, left=df.index, right=df.index)
    p_vol.ygrid.grid_line_color = None
    p_vol.toolbar_location = None
    
    # Plot - Altcoins daily returns smoothed
    df_returns_range = df_returns.loc[start:end]
    p_alts_smoothed = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns SMOOTHED", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(-8,10))
#                     y_range=(df_returns['median'].rolling(window=5, center=True).mean().min(),
#                              df_returns['median'].rolling(window=5, center=True).mean().max()))
    p_alts_smoothed.line(df_returns.index, df_returns['median'].rolling(window=5, center=True).mean(), line_width=1)
    p_alts_smoothed.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts_smoothed.toolbar_location = None
    
    # Plot - Altcoins daily returns
    df_returns_range = df_returns.loc[start:end]
    p_alts = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(df_returns_range['median'].min(), df_returns_range['median'].max()))
    p_alts.line(df_returns.index, df_returns['median'], line_width=1)
    #p_alts.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts.xaxis.visible = False
    
    p_alts.yaxis.axis_label = 'Daily returns in %'
    p_alts_smoothed.yaxis.axis_label = 'Daily returns in %'
    
    grid = gridplot([[p_alts], [p_alts_smoothed], [p], [p_vol]])
    
    return grid

In [13]:
start = '2017-7-27'
end = '2018-1-31'
grid = plot_btc_2(start, end)
show(grid)

In [24]:
def plot_btc_3(start, end):
    
    plot_width = 1200
    
    plot_height=600
    vol_height=100
    rsi_height=150
    
    df = btc_ohlcv.loc[start:end]
    start_dt  = parser.parse(start) 
    end_dt = parser.parse(end) 
    
    inc = df.close > df.open
    dec = df.open > df.close

    TOOLS = "save"  
    
    title = "Bitcoin 1D, from " + start + " to " + end 
    p = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height,
               title=title, x_range=Range1d(start=start_dt, end=end_dt), y_range=(0.8*df['low'].min(), 1.02*df['high'].max()))
    p.xaxis.visible = False
    p.min_border_top = 40
    
    # Add indicators
    p.line(df.index, df['sma50'], line_width=1, color='#66c2ff')
    p.line(df.index, df['sma200'], line_width=1, color='#ff6666')

    # Plot - Price chart
    p.toolbar_location = None
    p.segment(df.index, df.high, df.index, df.low, color="#808080")
    p.vbar(df.index[inc], 24 * 60 * 60 * 850, df.open[inc], df.close[inc], fill_color="#35a746",
           line_color="#35a746")
    p.vbar(df.index[dec], 24 * 60 * 60 * 850, df.open[dec], df.close[dec], fill_color="#d91813",
           line_color="#d91813")

    # Add marks for extreme volatile days
    df_down, df_up = high_vol_days(5, -5)
        
    day_range = (end_dt - start_dt).days
    font_size = str(round(3000/day_range)) + 'pt'
    for index, row in df_down.iterrows():
        p.triangle(x=index, y=0.92*row['low'], size=1000/day_range,
                   line_color="#ff0000", line_width=1, fill_color='#ff0000')
        p.text(x=index, y=0.92*row['low'] - 700, text=[row['median_str']], x_offset=2200/day_range,
               text_font_size=font_size, angle=1.57, text_color="#ff0000")
    for index, row in df_up.iterrows():
        p.triangle(x=index, y=0.92*row['low'], size=1000/day_range,
                   line_color="#00e600", line_width=1, fill_color='#00cc00')
        p.text(x=index, y=0.92*row['low'] - 700, text=[row['median_str']], x_offset=2200/day_range,
               text_font_size=font_size, angle=1.57, text_color="#00cc00") 
        
    # Add marks for historical events
    for index, row in df_events.iterrows():
        p.line(x=[index,index], y=[0.2*df['low'].min(), 2*df['high'].max()], color='#808080')
        offset = 7000
        if 'registration' in row['events']:
            offset = 7000
        if 'SegWit' in row['events']:
            offset = 9000
        if 'CBOE' in row['events']:
            offset = 18000
        if 'Korea' in row['events']:
            offset = 18000
        p.text(x=index, y=df['high'].max() - offset, text=[row['events']], x_offset=5000/day_range,
           text_font_size='10pt', angle=1.57, text_color="#808080")
        #if index = ''

    # Plot - Volume chart
    p_vol = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=vol_height,
               title=None, x_range=Range1d(start=start_dt, end=end_dt))
    p_vol.yaxis.visible = False
    p_vol.quad(bottom=0, top=df.volume, left=df.index, right=df.index)
    p_vol.ygrid.grid_line_color = None
    p_vol.toolbar_location = None
    
    # Plot - Altcoins daily returns smoothed
    df_returns_range = df_returns.loc[start:end]
    p_alts_smoothed = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns SMOOTHED", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(-5,3))
#                     y_range=(df_returns['median'].rolling(window=5, center=True).mean().min(),
#                              df_returns['median'].rolling(window=5, center=True).mean().max()))
    p_alts_smoothed.line(df_returns.index, df_returns['median'].rolling(window=5, center=True).mean(), line_width=1)
    p_alts_smoothed.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts_smoothed.toolbar_location = None
    
    # Plot - Altcoins daily returns
    df_returns_range = df_returns.loc[start:end]
    p_alts = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(df_returns_range['median'].min(), df_returns_range['median'].max()))
    p_alts.line(df_returns.index, df_returns['median'], line_width=1)
    #p_alts.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts.xaxis.visible = False
    
    p_alts.yaxis.axis_label = 'Daily returns in %'
    p_alts_smoothed.yaxis.axis_label = 'Daily returns in %'
    
    grid = gridplot([[p_alts], [p_alts_smoothed], [p], [p_vol]])
    
    return grid

In [25]:
start = '2018-1-1'
end = '2018-11-10'
grid = plot_btc_3(start, end)
show(grid)

In [38]:
def plot_btc_4(start, end):
    
    plot_width = 800
    
    plot_height=500
    vol_height=100
    rsi_height=150
    
    df = btc_ohlcv.loc[start:end]
    start_dt  = parser.parse(start) 
    end_dt = parser.parse(end) 
    
    inc = df.close > df.open
    dec = df.open > df.close

    TOOLS = "save"  
    
    title = "Bitcoin 1D, from " + start + " to " + end 
    p = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height,
               title=title, x_range=Range1d(start=start_dt, end=end_dt), y_range=(0.9*df['low'].min(), 1.02*df['high'].max()))
    p.xaxis.visible = False
    p.min_border_top = 40
    
    # Add indicators
    p.line(df.index, df['sma50'], line_width=1, color='#66c2ff')
    p.line(df.index, df['sma200'], line_width=1, color='#ff6666')

    # Plot - Price chart
    p.toolbar_location = None
    p.segment(df.index, df.high, df.index, df.low, color="#808080")
    p.vbar(df.index[inc], 24 * 60 * 60 * 600, df.open[inc], df.close[inc], fill_color="#35a746",
           line_color="#35a746")
    p.vbar(df.index[dec], 24 * 60 * 60 * 600, df.open[dec], df.close[dec], fill_color="#d91813",
           line_color="#d91813")

    # Add marks for extreme volatile days  
    df_down, df_up = high_vol_days(3, -3)
    
    day_range = (end_dt - start_dt).days
    font_size = str(round(1600/day_range)) + 'pt'
    for index, row in df_down.iterrows():
        p.triangle(x=index, y=0.97*row['low'], size=1000/day_range,
                   line_color="#ff0000", line_width=1, fill_color='#ff0000')
        p.text(x=index, y=0.97*row['low'] - 300, text=[row['median_str']], x_offset=1200/day_range,
               text_font_size=font_size, angle=1.57, text_color="#ff0000")
    for index, row in df_up.iterrows():
        p.triangle(x=index, y=0.97*row['low'], size=1000/day_range,
                   line_color="#00e600", line_width=1, fill_color='#00cc00')
        p.text(x=index, y=0.97*row['low'] - 300, text=[row['median_str']], x_offset=1200/day_range,
               text_font_size=font_size, angle=1.57, text_color="#00cc00") 
        
    # Add marks for historical events
    for index, row in df_events.iterrows():
        p.line(x=[index,index], y=[0.2*df['low'].min(), 2*df['high'].max()], color='#808080')
        offset = 7000
        if 'registration' in row['events']:
            offset = 7000
        if 'SegWit' in row['events']:
            offset = 9000
        if 'CBOE' in row['events']:
            offset = 18000
        if 'Korea' in row['events']:
            offset = 18000
        p.text(x=index, y=df['high'].max() - offset, text=[row['events']], x_offset=5000/day_range,
           text_font_size='10pt', angle=1.57, text_color="#808080")
        #if index = ''

    # Plot - Volume chart
    p_vol = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=vol_height,
               title=None, x_range=Range1d(start=start_dt, end=end_dt))
    p_vol.yaxis.visible = False
    p_vol.quad(bottom=0, top=df.volume, left=df.index, right=df.index)
    p_vol.ygrid.grid_line_color = None
    p_vol.toolbar_location = None
    
    # Plot - Altcoins daily returns smoothed
    df_returns_range = df_returns.loc[start:end]
    p_alts_smoothed = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns SMOOTHED", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(-2.5,1.5))
#                     y_range=(df_returns['median'].rolling(window=5, center=True).mean().min(),
#                              df_returns['median'].rolling(window=5, center=True).mean().max()))
    p_alts_smoothed.line(df_returns.index, df_returns['median'].rolling(window=5, center=True).mean(), line_width=1)
    p_alts_smoothed.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts_smoothed.toolbar_location = None
    
    # Plot - Altcoins daily returns
    df_returns_range = df_returns.loc[start:end]
    p_alts = figure(tools=TOOLS, x_axis_type="datetime", plot_width=plot_width, plot_height=150,
                   title="Altcoins daily percentage returns", 
                    x_range=Range1d(start=start_dt, end=end_dt), 
                    y_range=(df_returns_range['median'].min(), df_returns_range['median'].max()))
    p_alts.line(df_returns.index, df_returns['median'], line_width=1)
    #p_alts.line(x=[start_dt,end_dt], y=[0,0], color='#333333')
    p_alts.xaxis.visible = False
    
    p_alts.yaxis.axis_label = 'Daily returns in %'
    p_alts_smoothed.yaxis.axis_label = 'Daily returns in %'
    
    grid = gridplot([[p_alts], [p_alts_smoothed], [p], [p_vol]])
    
    return grid

In [39]:
start = '2018-11-1'
end = '2019-4-18'
grid = plot_btc_4(start, end)
show(grid)